# Bering Sea with flexible biology: application setup

This script builds a working ROMS application environment with the following components:

 - Grid: Bering10K
 - Ocean model: nonlinear
 - Ice model: Budgell (Hedstrom version)
 - Biogeochemical models: 
   - BEST_NPZ
   - BIO_COBALT
   - BIO_BANAS

In [1]:
import os
import sys
import yaml
from collections import OrderedDict

sys.path.insert(0, '/Users/kakearney/Documents/PythonCodeKAK/roms_input_wrangler')

import varinfo
import standard_input as si

## Header file

The header file combines the defaults for the Bering_10K application along with all biological module options.  The bio-dependent flags are wrapped in a conditional such that the specific module to be compiled is turned on by the compilation script (build_roms.bash).  This allows the same header file to be used for all app compilation variants.  Ideally, the final header file should not be unless major changes are made to the Bering_10K ocean model configuration or one of the biogeochemical models.

In [2]:
newfile = "test_bering_10k.h"

berapps = "../.."
header_subfiles = [os.path.join(berapps, "subApps", "BERING_10K_new", "bering_10k.h"), 
                   os.path.join(berapps, "subApps", "BIO_COBALT",     "cobalt_cpp.h"), 
                   os.path.join(berapps, "subApps", "BEST_NPZ",       "bestnpz_cpp.h"),
                   os.path.join(berapps, "subApps", "BIO_BANAS",      "banas_cpp.h")]

# Concatenate sub-files into new header file

with open(newfile, 'w') as out:

    for file in header_subfiles:
        with open(file, 'r') as f:
            contents = f.read()
            out.write(contents)

## varinfo.dat

We build several different versions of the varinfo.dat file, to be used with different compiled versions.  The physics-only option combines I/O details for ocean and ice variables.  Each biological model adds biological I/O details to that physics-only option.  These are kept separate to ensure no variable conflicts occur between BGC options.

In [3]:
# Read default ocean I/O (note: variables organized into many topic arrays)

with open(os.path.join(berapps, "subApps", "ocean", "default_io.yaml"), 'r') as f:
    Vphys = yaml.load(f)

# Ice and BGC I/O files only hold a single metadata array, so read with varinfo.readfile wrapper to extract just that
    
Vice = varinfo.readfile(os.path.join(berapps, "subApps", "IceBudgell", "ice_io.yaml"),     type="yaml")

Vbes = varinfo.readfile(os.path.join(berapps, "subApps", "BEST_NPZ",   "bestnpz_io.yaml"), type="yaml")
Vcob = varinfo.readfile(os.path.join(berapps, "subApps", "BIO_COBALT", "cobalt_io.yaml"),  type="yaml")
Vban = varinfo.readfile(os.path.join(berapps, "subApps", "BIO_BANAS",  "banas_io.yaml"),   type="yaml")

# Combine physics variables (splitting based on where bio variables will be inserted)

V1 = Vphys['state_variables'] + \
     Vphys['wet_dry_masks'] + \
     Vphys['detided'] + \
     Vphys['momentum'] + \
     Vphys['tracer_flux'] + \
     Vphys['mixing'] + \
     Vphys['boundary'] + \
     Vphys['atmos_forcing'] + \
     Vphys['tidal_forcing'] + \
     Vphys['river_forcing'] + \
     Vphys['boundary_layer'] + \
     Vphys['climatology'] + \
     Vphys['adjoint'] + \
     Vphys['tangent'] + \
     Vphys['observation_vars'] + \
     Vphys['passive'] + \
     Vphys['sediment']
        
V2 = Vphys['quadratic'] + \
     Vphys['diagnostic'] + \
     Vphys['forward_trajectory'] + \
     Vphys['assimilation']

    
# Rename bottom drag coefficient (in sediment) to match Bering10K grid file (rdrag -> rdrag_grid)

for ii in V1:
    if ii['index_code'] == "idragL":
        ii['variable'] = "rdrag_grid"
        
# Combine ocean and ice I/O

Vbase = V1 + V2 + Vice

# Combine with bio options.  Note that bio state variables need to preceed any variable 
# that includes (itrc) in their ID (e.g. tracer2 and others in quadratic set). I opt to 
# place all tracers together just before quadratic terms, and other bio variables at the end.

trc = [];
oth = []
for ii in Vbes:
    if ii['index_code'].startswith('idTvar(') | \
       ii['index_code'].startswith('idBeTvar(') | \
       ii['index_code'].startswith('idBeTvar('):
        trc.append(ii)
    else:
        oth.append(ii)
        
Vbes = V1 + trc + V2 + Vice + oth


trc = [];
oth = []
for ii in Vcob:
    if ii['index_code'].startswith('idTvar(') | \
       ii['index_code'].startswith('idBeTvar(') | \
       ii['index_code'].startswith('idBeTvar('):
        trc.append(ii)
    else:
        oth.append(ii)
        
Vcob = V1 + trc + V2 + Vice + oth

trc = [];
oth = []
for ii in Vban:
    if ii['index_code'].startswith('idTvar(') | \
       ii['index_code'].startswith('idBeTvar(') | \
       ii['index_code'].startswith('idBeTvar('):
        trc.append(ii)
    else:
        oth.append(ii)
        
Vban = V1 + trc + V2 + Vice + oth

# Write to files (classic format)

varinfo.writefile(Vbase, "test_varinfo_phys.dat",    type="classic")
varinfo.writefile(Vbes,  "test_varinfo_bestnpz.dat", type="classic")
varinfo.writefile(Vcob,  "test_varinfo_cobalt.dat",  type="classic")
varinfo.writefile(Vban,  "test_varinfo_banas.dat",   type="classic")

## Input file templates

In [4]:
with open(os.path.join(berapps, "subApps", "BERING_10K_new","ocean_bering10k_new.in.yaml"), 'r') as f:
    test = si.ordered_load(f)


In [14]:
# bpardict testing

# paramfile = os.path.join(berapps, "subApps", "BIO_BANAS", "banas_parameters.yaml")
# iofile = os.path.join(berapps, "subApps", "BIO_BANAS", "banas_io.yaml")

paramfile = os.path.join(berapps, "subApps", "BEST_NPZ", "bestnpz_parameters.yaml")
iofile = os.path.join(berapps, "subApps", "BEST_NPZ", "bestnpz_io.yaml")

# Read parameters

P = yaml.load(open(paramfile))
Io = yaml.load(open(iofile))
Io = Io['metadata']

# Start dict with parameters

a = OrderedDict();
a['Lbiology'] = True
a['BioIter'] = 1

# print(yaml.dump({'Lbiology': True}))
# print(yaml.dump({'BioIter': 1}))

for ii in P:
    a[ii['param']] = ii['value']
    

    
# # Add extras

# nbt = 0
# nben = 0
# nice = 0
# for ii in Io:
#     if ii['index_code'].startswith('idTvar('):
#         nbt+=1
#     elif ii['index_code'].startswith('idBeTvar('):
#         nben+=1
#     elif ii['index_code'].startswith('idIceTvar('):
#         nice+=1

yaml.add_representer(OrderedDict, lambda dumper, data: dumper.represent_mapping('tag:yaml.org,2002:map', data.items()))

print(yaml.dump(a, default_flow_style=False))

        

Lbiology: true
BioIter: 1
PARfrac: 0.42
k_ext: 0.034
k_chlA: 0.0518
k_chlB: 0.428
k_chlC: 0.0363
k_sed1: 2.833
k_sed2: -1.079
xi: 0.0126
ccr: 65.0
ccrPhL: 25.0
FeC: 0.0001667
DiS: 0.5
DiL: 1.0
DpS: 0.0275
DpL: 0.0275
alphaPhS: 5.6
alphaPhL: 2.2
k1PhS: 1.0
k1PhL: 2.0
k2PhS: 0.5
k2PhL: 2.0
FeCritPS: 2.0
FeCritPL: 2.0
kfePhS: 0.3
kfePhL: 1.0
fpPhSMZL: 1.0
fpPhLMZL: 0.2
fpPhSCop: 0.8
fpPhLCop: 0.7
fpMZLCop: 0.5
fpPhSNCa: 0.1
fpPhLNCa: 1.0
fpMZLNCa: 1.0
fpPhSEup: 1.0
fpPhLEup: 1.0
fpMZLEup: 1.0
fpCopEup: 0.2
fpDetEup: 0.4
fpDetEupO: 0.0
fpCopJel: 1.0
fpNCaJel: 1.0
fpEupJel: 1.0
eMZL: 0.4
eCop: 0.4
eNCa: 0.3
eEup: 0.3
eJel: 0.069
Q10MZL: 2.0
Q10Cop: 1.7
Q10NCa: 1.6
Q10Eup: 1.5
Q10Jele: 2.4
Q10MZLT: 5.0
Q10CopT: 5.0
Q10NCaT: 5.0
Q10EupT: 5.0
Q10JelTe: 10.0
fMZL: 20.0
fCop: 30.0
fNCa: 30.0
fEup: 40.0
fJel: 0.01
gammaMZL: 0.7
gammaCop: 0.7
gammaNCa: 0.7
gammaEup: 0.7
gammaJel: 1.0
mPhS: 0.01
mPhL: 0.01
mMZL: 0.0
mpredMZL: 0.01
mpredCop: 0.05
mpredNCa: 0.05
mpredEup: 0.05
mpredJel: 0.006
wPhS: 0

In [13]:
P

{'metadata': [{'descrip': 'Fraction of irradiance that is photosynthetically available (PAR)',
   'param': 'PARfrac',
   'unit': 'unitless',
   'value': 0.42},
  {'descrip': 'Clear-water attenuation coefficient',
   'param': 'k_ext',
   'unit': 'm^-1',
   'value': 0.034},
  {'descrip': 'Chlorophyll attenuation coefficient, factor',
   'param': 'k_chlA',
   'unit': 'm^-1',
   'value': 0.0518},
  {'descrip': 'Chlorophyll attenuation coefficient, exponent',
   'param': 'k_chlB',
   'unit': 'unitless',
   'value': 0.428},
  {'descrip': 'Other material (CDOM,sediment,etc.) attenuation coefficient',
   'param': 'k_chlC',
   'unit': 'm^-1',
   'value': 0.0363},
  {'descrip': 'Depth-based attenuation coefficient, factor',
   'param': 'k_sed1',
   'unit': 'm^-1',
   'value': 2.833},
  {'descrip': 'Depth-based attenuation coefficient, exponent',
   'param': 'k_sed2',
   'unit': 'unitless',
   'value': -1.079},
  {'descrip': 'Nitrogen:Carbon ratio',
   'param': 'xi',
   'unit': 'mmol N / mg C',
 